In [ ]:
!pip install fastapi uvicorn nest-asyncio 

!apt install ffmpeg
!pip install yt-dlp

from google.colab import drive

drive.mount("/content/drive")

# 保存用ディレクトリ作成
import os

path = "/content/drive/MyDrive/MediaKnife"
os.makedirs(path, exist_ok=True)

In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.staticfiles import StaticFiles

app = FastAPI()


# ファイルの一覧を取得
@app.get("/upload")
async def read_upload_file():
    files = os.listdir(f"{path}/uploads")
    return {"files": files}


from pydantic import BaseModel


class ytDto(BaseModel):
    url: str


from yt_dlp import YoutubeDL


# 動画ダウンロード
@app.post("/yt-dlp")
async def yt_dlp(urlDto: ytDto):
    url = urlDto.url

    # ディレクトリが存在しなければ作成
    save_dir = f"{path}/uploads"
    os.makedirs(save_dir, exist_ok=True)

    # 動画の情報を抽出
    ydl = YoutubeDL()
    info = ydl.extract_info(url, download=False)
    title = info.get("title", None)

    safe_filename = f"{save_dir}/{title}.mp4"
    # ファイル保存
    save_path = f"{save_dir}/{safe_filename}"

    # yt-dlpでダウンロード
    # !yt-dlp -f "bestvideo[ext=mp4]+bestaudio[ext=m4a]" --merge-output-format mp4 --no-mtime -o "$save_path" "$url"
    ydl_opts = {
        "format": "bestvideo[ext=mp4]+bestaudio[ext=m4a]",
        "merge_output_format": "mp4",
        "no_mtime": True,
        "outtmpl": save_path,
    }
    with YoutubeDL(ydl_opts) as ydl:
        result = ydl.download([url])
        if result == 0:
            return {"filename": safe_filename}

    return {"filename": safe_filename}


@app.post("/upload")
async def create_upload_file(file: UploadFile = File(...)):
    # ファイル名生成
    safe_filename = f"{file.filename}"

    # ディレクトリが存在しなければ作成
    save_dir = f"{path}/uploads"
    os.makedirs(save_dir, exist_ok=True)

    # ファイル保存
    save_path = f"{save_dir}/{safe_filename}"

    with open(save_path, "wb+") as f:
        f.write(file.file.read())

    return {"filename": safe_filename}


# TODO: distコンテンツを取得する処理
app.mount("/", StaticFiles(directory="dist", html=True), name="static")

import nest_asyncio

nest_asyncio.apply()


from google.colab import output

output.serve_kernel_port_as_iframe(8000, height=900)


import uvicorn

uvicorn.run(app, port=8000, host="0.0.0.0")